In [1]:
# Import Libraries
import numpy as np

from project_utils.bb_model         import BB_Model
from project_utils.feature_stats    import Feature_Statistics_C
from project_utils.feature_stats    import Regression_Container
from project_utils.acq_data_capture import Acq_Data_nD
from project_utils.surroagte_gp     import Surrogate_GP



In [2]:
Num_Points    = 6
Title         = 'parkinsons'
FUR_Weight    = 0.1
N_consistancy = 25
consist_bound = 0.3

In [3]:
if isinstance(FUR_Weight, float):
    FUR_Weight = [FUR_Weight, 1.0 - FUR_Weight]
    
print(FUR_Weight)

[0.1, 0.9]


In [4]:
Title = Title + ' W:'+ str(FUR_Weight)

In [5]:
# Load Forrester model

Class_data = BB_Model(dataset = 'parkinsons')
                      
#Class_data.df().head()

In [6]:
Class_data.MPL()

C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.8205128205128205


MLPClassifier(random_state=RandomState(MT19937) at 0x20DF03A9840)

In [7]:
Class_data.Random_Forest()

0.8974358974358975


RandomForestClassifier(random_state=RandomState(MT19937) at 0x20DF03A9840)

In [8]:
#Class_data.GP()

In [9]:
Class_data.L_Regression()

C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8205128205128205


LogisticRegression()

In [10]:
print(Class_data.get_features())

['MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)', 'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5', 'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'RPDE', 'DFA', 'spread1', 'spread2', 'D2', 'PPE']


In [11]:
#Default settings
from unravel_2.ur_model import UR_Model

X_train, X_test, y_train, y_test = Class_data.get_TT_data()

#Initializing the explainer object
URM_Exp = UR_Model(bbox_model           = Class_data.get_Random_Forest(),
                   train_data           = X_train,
                   feature_names        = Class_data.get_features(),
                   categorical_features = Class_data.get_catagorical(),
                   mode                 = Class_data.get_mode(),
                   sampling_optimize    = "opt")

#sur_GP = Surrogate_GP()


In [12]:
#I2nd explainer object
URM_Exp_2 = UR_Model(bbox_model           = Class_data.get_L_Regression(),
                     train_data           = X_train,
                     feature_names        = Class_data.get_features(),
                     categorical_features = Class_data.get_catagorical(),
                     mode                 = Class_data.get_mode(),
                     sampling_optimize    = "opt")



In [13]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
print('Mean 0:', mean)
print('Std 0: ', std)


Mean 0: [ 1.56864705e+02  1.98275212e+02  1.18726763e+02  6.54160256e-03
  4.57243590e-05  3.52032051e-03  3.60557692e-03  1.05618590e-02
  3.06094231e-02  2.92230769e-01  1.61391026e-02  1.83731410e-02
  2.49955128e-02  4.84179487e-02  2.75613462e-02  2.15662244e+01
  5.03830750e-01  7.15649487e-01 -5.64149228e+00  2.28559019e-01
  2.40441853e+00  2.09741660e-01]
Std 0:  [4.30045001e+01 9.08405138e+01 4.57994639e+01 5.26236328e-03
 3.75978337e-05 3.22478889e-03 2.97213582e-03 9.67464336e-03
 1.94893853e-02 2.03747545e-01 1.04278080e-02 1.22846771e-02
 1.80052208e-02 3.12831813e-02 4.41632156e-02 4.54018934e+00
 1.05410902e-01 5.49328155e-02 1.14850257e+00 8.66347241e-02
 3.88033992e-01 9.47359622e-02]


In [ ]:
# Generate explantion data for Random Forest

Class_Stats   = Feature_Statistics_C(feature_names = Class_data.get_features(),
                                     mode          = Class_data.get_mode(),
                                     classes       = ['Not-Diabetic', 'Diabetic'],
                                     X_train_std   = Class_data.get_X_train_std(),
                                     N_consistancy = N_consistancy)

Class_Stats_2 = Feature_Statistics_C(feature_names = Class_data.get_features(),
                                     mode          = Class_data.get_mode(),
                                     classes       = ['Not-Diabetic', 'Diabetic'],
                                     X_train_std   = Class_data.get_X_train_std())


if Num_Points > X_test.shape[0]: Num_Points = X_test.shape[0]

elif Num_Points == 0:            Num_Points = X_test.shape[0]

for test_instance in range(Num_Points):

    print('INSTANCE: ', test_instance)
       
    data_row  = np.array([X_test[test_instance]])
    
#    data_row  = np.array([0.4])

#    print('data row ',data_row)
    
    URM_Exp.explain(X_init      = data_row.reshape(1, -1),
                    Dimension   = 'Multi D',
                    kernel_type = "Matern_3",
                    max_iter    = 20,
                    alpha       = "FUR_W",
                    #jitter=5,
                    interval    = 1,
                    weight      = FUR_Weight)
       
    f_predict = Class_data.get_Random_Forest().predict_proba(data_row.reshape(1, -1))

    e_predict = URM_Exp.exp_predict(data_row.reshape(1, -1))
    e_predict = np.array(e_predict).ravel()
    
    acq_data = URM_Exp.get_acq_data()
    
#    X, y = URM_Exp.get_surrogate_data()

#    e_predict = sur_GP.predict(X, y)
#    e_predict = sur_GP.predict(URM_Exp.get_surrogate_data())

#    print(f_predict, e_predict, acq_data.get_fe_x0())
    
    scores = URM_Exp.permutation_importance()
    
    consistancy = URM_Exp.Y_Consistancy(N_points = N_consistancy, std_bound = consist_bound)
    
    URM_Exp.del_1_rel()
    
    del_1_var = URM_Exp.get_del_1_variance()
    
    Class_Stats.Add_Sample(sample_scores = scores,
                         outcome       = y_test[test_instance],
                         X_row         = data_row.reshape(1, -1),
                         f_prediction  = f_predict,
                         e_prediction  = e_predict.ravel(),
                         feopt         = acq_data.get_fe_x0(),
                         consistancy   = consistancy,
                         del_1_var     = del_1_var)
    
    
    ############################################################################################
    
    URM_Exp_2.explain(X_init      = data_row.reshape(1, -1),
                      Dimension   = 'Multi D',
                      kernel_type = "Matern_3",
                      max_iter    = 20,
                      alpha       = "FUR_W",
                      #jitter=5,
                      interval    = 1,
                      weight      = FUR_Weight)
       
    f_predict = Class_data.get_L_Regression().predict_proba(data_row.reshape(1, -1))

    e_predict = URM_Exp_2.exp_predict(data_row.reshape(1, -1))
    e_predict = np.array(e_predict).ravel()
    
    scores = URM_Exp_2.permutation_importance()
    
    Class_Stats_2.Add_Sample(sample_scores = scores,
                           outcome       = y_test[test_instance],
                           X_row         = data_row.reshape(1, -1),
                           f_prediction  = f_predict,
                           e_prediction  = e_predict.ravel(),
                           feopt         = None)
    


INSTANCE:  0
Acq_Data_nD
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT


C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 1000000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 1 of parameter length_scale is close to the specified upper bound 1000000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 2 of parameter length_scale is close to the specified upper bound 1000000000.0. Increasing the bound and calling fit again may find a better value.
  warn

Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Optimize FAIL 9 : 0 False


C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 1000000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 1 of parameter length_scale is close to the specified upper bound 1000000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 2 of parameter length_scale is close to the specified upper bound 1000000000.0. Increasing the bound and calling fit again may find a better value.
  warn

Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT


...

Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT


C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 1000000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 1 of parameter length_scale is close to the specified upper bound 1000000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 2 of parameter length_scale is close to the specified upper bound 1000000000.0. Increasing the bound and calling fit again may find a better value.
  warn

Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT


C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 1000000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 1 of parameter length_scale is close to the specified upper bound 1000000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 2 of parameter length_scale is close to the specified upper bound 1000000000.0. Increasing the bound and calling fit again may find a better value.
  warn

ADD ----> [[0. 1.]] [1.00000000e+00 2.18219353e-06]
Acq_Data_nD
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Opt iteration fail:  STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
print(f_predict)
print(e_predict)


In [ ]:
Class_Stats.Feature_Counts(max_features=20, scaled=True, threshold=0.05)


In [ ]:
Class_Stats.Frequency_Plot()


In [ ]:
Class_Stats.Class_Fidelity()

In [ ]:
Class_Stats.Jaccard_Values()

In [ ]:
Class_Stats.Box_Plot()


In [ ]:
Class_Stats.View_Explanation(instance = 0)

In [ ]:
scores = URM_Exp.permutation_importance()
    
print(scores)

In [ ]:
scores_k = URM_Exp.KL_imp()

print(scores_k)

In [ ]:
scores_v = URM_Exp.Var_imp()

print(scores_v)

In [ ]:
scores_d1 = URM_Exp.del_1_rel()

print(scores_d1)

In [ ]:
scores_d2 = URM_Exp.del_2_rel()

print(scores_d2)

In [ ]:
lin_scores = URM_Exp.Lin_scores()

print(lin_scores)

In [ ]:
URM_Exp.plot_scores(Title)

In [ ]:
print(URM_Exp.get_exp_L())


In [ ]:
acq_data.Create_BB_plot()

In [ ]:
acq_data.plot_point(0)

In [ ]:
acq_data.plot_point(5)

In [ ]:
acq_data.plot_point(10)

In [ ]:
acq_data.plot_point(15)

In [ ]:
acq_data.plot_point(19)

In [ ]:
acq_data.plot_all()

In [ ]:
acq_data.plot_t1_t2(0)

In [ ]:
acq_data.plot_t1_t2(1)

In [ ]:
acq_data.plot_t1_t2(2)

In [ ]:
acq_data.plot_t1_t2(3)

In [ ]:
acq_data.plot_t1_t2(4)

In [ ]:
acq_data.plot_t1_t2(5)

In [ ]:
acq_data.plot_t1_t2(6)

In [ ]:
acq_data.plot_t1_t2(7)

In [ ]:
acq_data.plot_t1_t2(8)

In [ ]:
acq_data.plot_t1_t2(9)

In [ ]:
acq_data.plot_t1_t2(10)

In [ ]:
acq_data.plot_t1_t2(11)

In [ ]:
acq_data.plot_t1_t2(12)

In [ ]:
acq_data.plot_t1_t2(13)

In [ ]:
acq_data.plot_t1_t2(14)

In [ ]:
acq_data.plot_t1_t2(15)

In [ ]:
acq_data.plot_t1_t2(16)

In [ ]:
acq_data.plot_t1_t2(17)

In [ ]:
acq_data.plot_t1_t2(18)

In [ ]:
acq_data.plot_t1_t2(19)

In [ ]:
Class_Stats.Class_Fidelity()

print('Fidelity: ', Class_Stats.fidelity)

In [ ]:
Class_Stats.Jaccard_Values(top_k=4)

print('Jaccard: ', Class_Stats.jaccard_similarities)

In [ ]:
Class_Stats.Consistancy(std_bound = consist_bound, plot=True, title = Title)

In [ ]:
Class_Stats.Compare_Models (model_b = Class_Stats_2)

In [ ]:
Class_Stats.Calibration()

In [ ]:
Class_Stats.delete_one()

In [ ]:
Class_Stats.Results_Summary(Title)